# 使用装饰器对已存在的类添加功能失败的尝试

最近在yotsuha项目中，我打算使用Python内置的decorator功能尝试对已存在的类添加功能。但是，由于类内部无法访问由装饰器添加的功能，因而失败。现在整个项目将寻找另外的一种方法解决代码重复问题。

## 动机
现在项目里，有两个类`YandereCrawler`与`AsyncRequestScheduler`。前者实现了一个爬虫类，用于爬取`yande.re`网站上面的图片，而后者是利用`requests`库实现的一个可以按照指定的时间间隔发送网络请求的类。现在的工作是想将这两个类中有关显示的部分重构为一个单独的类。我打算使用观察者模式，当上述两者发生动作时，就发送一个通知给所有的观察者。通过一个API可以调取需要的数据。

但是在实现中，我发现了这两个类中间出现了粘贴复制式的重复：

    class YandereCralwer:
        def __init__(self, ...):
            # Other initalization
            ...
            self._observable = Observable
        
        def observe(self, observer_func):
            self._observable.register(observer_func)
            
        def _notify(self, event):
            self._observable.notify({
                'event': event
            })
        # Other methods
        ...
        
    class AsyncRequestScheduler:
        def __init__(self, ...):
        # Other initalization
        ...
        self._observable = Observable
        
        def observe(self, observer_func):
            self._observable.register(observer_func)
            
        def _notify(self, event):
            self._observable.notify({
                'event': event
            })
        # Other methods
        ...
        
当出现这种情况时，显然应该针对两个类都是Observable的进行一定的重构。

## 失败的方法
为了不修改原有的代码，并且能够添加上我所需要的功能，我使用了`class decorator`。它是一种语法糖，可以在被装饰的class的定义，之后动态地绑定到被装饰的class中。如：
    
    @decorator
    class SomeClass:
        ...
        
    class SomeClass:
        ...
    SomeClass = decorator()(SomeClass)
    
上述两种形式在语义上是等价的。

《Learning Python》中提出了一种通过`class decorator`增强原有类的方法，其中`decorator`的写法如下：

    def decorator(class_):
        class Wrapper:
            def __init__(self, *args, **kwargs):
                self.wrapped = class_(*args, **kwargs)
            
            def argument(self):
                print('calling argument.')
            
            def __getattr__(self, name):
                return getattr(self.wrapped, name)
                
        return Wrapped

这种方法可以通过使用`decorator`装饰某个类，将`argument`方法加载到被装饰的类中。同时保留原有类的所有方法（通过`__getattr__`）。这个方法在下面的cell中，我们将会演示这种方法的效果。

In [5]:
def decorator(class_):
    class Wrapper:
        def __init__(self, *args, **kwargs):
            self.wrapped = class_(*args, **kwargs)

        def argument(self):
            print('calling argument.')

        def __getattr__(self, name):
            return getattr(self.wrapped, name)
    return Wrapper

@decorator
class A:
    def print_info(self):
        print('Print from A')
        
    def cannot_call_from_here(self):
        print('call from A')
        self.argument()

instance = A()
instance.print_info()
instance.argument()
instance.cannot_call_from_here()

Print from A
calling argument.
call from A


AttributeError: 'A' object has no attribute 'argument'

从上面可以看到，**从外部**可以调用装饰器所带来的新方法。但是**从内部**，我们原有的类却不知道装饰器的存在。当然，这个结论是合理的。*被装饰的类当然不知道会有什么类去装饰自己。*
因此，通过装饰器这种方式添加对外的接口是可行的。但是，被装饰类无法调用，这不符合我们的需求。